In [9]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical
import pandas as pd
import numpy as np 
import json
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import sentencepiece as spm
import os 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [10]:
df = pd.read_json(r'dataset/train_mr.jsonl', lines=True)
df = pd.json_normalize(df['row'])

In [11]:
sp = spm.SentencePieceProcessor()
sp.load('my_tokenizer.model')

df['tokens_input'] = df['input'].apply(lambda x: sp.encode(x, out_type=str))
df['ids_input'] = df['input'].apply(lambda x: sp.encode(x, out_type=int))

df['tokens_target'] = df['target'].apply(lambda x: sp.encode(x, out_type=str))
df['ids_target'] = df['target'].apply(lambda x: sp.encode(x, out_type=int))

In [12]:
X = pad_sequences(df['ids_input'], padding='post')
# For targets, we only take the first token for simplicity
y = np.array([t[0] for t in df['ids_target']])

vocab_size = sp.get_piece_size()
y = to_categorical(y, num_classes=vocab_size)

In [13]:
model = Sequential([
    Embedding(vocab_size, 64),
    LSTM(128),
    Dense(vocab_size, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=30, verbose=1)

Epoch 1/30
212/216 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.0107 - loss: 9.5799

KeyboardInterrupt: 